In [1]:
from netCDF4 import Dataset
import rasterio
from affine import Affine
from pyproj import Proj, transform

In [2]:
test = Dataset('./data/test.nc','w')

In [3]:
fl = './data/haines_tiffs/gas_production_1980.tif'

with rasterio.open(fl) as r:
    T0 = r.affine # upper-left pixel corner affine transform
    A = r.read(1) # get pixel values
    cs = T0[0] # get cell size
# generate rows and columns
cols,rows = np.meshgrid(np.arange(A.shape[1]), np.arange(A.shape[0]))

In [6]:
r.crs['units'] = 'us-ft'
r.crs['to_meter'] = 0.304800609601219

In [8]:
p1 = Proj(r.crs, preserve_units=True)

In [10]:
T1 = T0 * Affine.translation(0.5,0.5) # get affine transform for pixel centers

rc2en = lambda r,c: (c,r) * T1 # function to convert row/col index to easting/northing at centre

eastings,northings  = np.vectorize(rc2en)(rows,cols)

# project all longitudes and latitutes
p2 = Proj(proj = 'latlong',datum='WGS84')
longs,lats = transform(p1,p2, eastings,northings)

In [11]:
print(round(eastings.min()-(cs/2),3))
print(round(eastings.max()+(cs/2),3))
print(round(northings.min()-(cs/2),3))
print(round(northings.max()+(cs/2),3))

-1127952.76
1332527.24
17164698.164
19530138.164


In [15]:
print(round(longs.min(),3))
print(round(longs.max(),3))
print(round(lats.min(),3))
print(round(lats.max(),3))

-108.79
-98.341
44.402
50.93


In [15]:
easting.shape

(466,)

In [16]:
p1.srs

'+proj=lcc +lat_1=33 +lat_2=45 +lat_0=0 +lon_0=-104 +x_0=0 +y_0=0 +datum=NAD83 +units=m +no_defs=True +to_meter=0.304800609601219 '

In [17]:
longs.min()

-179.99943490019342

In [18]:
longs.max()

179.99944915525055

In [19]:
lats.min()

44.698302361674358

In [20]:
lats.max()

65.955221782136661